In [19]:
import gspread
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [20]:
service_account = gspread.service_account(filename='./config/expense-tracker-358105-199d116b0a6d.json')
workbook = service_account.open('badminton_tracking')
worksheet = workbook.get_worksheet(0)

In [21]:
df = pd.DataFrame(worksheet.get_all_records()).drop(["Timestamp", "result"], axis=1)

df.columns = ["date", "team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2", "points_team_1", "points_team_2", "venue"]

df['winner'] = np.where(df.points_team_1 > df.points_team_2, 'team_1', 'team_2')
df['margin'] = abs(df.points_team_1 - df.points_team_2)
df['total_points_per_game'] = df["points_team_1"] + df["points_team_2"]

df = df.applymap(lambda x: f'{x}'.lower().strip() if isinstance(x, str) else x)

In [22]:
df['point_bins'] = pd.cut(
    df['total_points_per_game'],
    [0, 30, 35, 40, 45, float("inf")],
    right=False,
    labels=['< 30', '30 - 35', '35 - 40', '40 - 45', "> 45"]
)

### Game length stats
_by venue and date_

In [23]:
df.groupby(["venue", "point_bins"]).agg(**{
    "total_games": pd.NamedAgg("date", "count")
}).reset_index()

,venue,point_bins,total_games
0,infinity badminton academy,< 30,0
1,infinity badminton academy,30 - 35,2
2,infinity badminton academy,35 - 40,8
3,infinity badminton academy,40 - 45,6
4,infinity badminton academy,> 45,0
5,isro match point,< 30,0
6,isro match point,30 - 35,5
7,isro match point,35 - 40,20
8,isro match point,40 - 45,4
9,isro match point,> 45,3


In [24]:
df.groupby(["point_bins"]).agg(**{
    "total_games": pd.NamedAgg("date", "count")
})

,total_games
point_bins,
< 30,0
30 - 35,12
35 - 40,51
40 - 45,22
> 45,3


In [25]:
df.groupby(["point_bins", "venue"]).agg(**{
        "total_games": pd.NamedAgg("date", "count")
    }).reset_index()

,point_bins,venue,total_games
0,< 30,infinity badminton academy,0
1,< 30,isro match point,0
2,< 30,match point - gublaala,0
3,< 30,prakash badminton academy,0
4,30 - 35,infinity badminton academy,2
5,30 - 35,isro match point,5
6,30 - 35,match point - gublaala,3
7,30 - 35,prakash badminton academy,2
8,35 - 40,infinity badminton academy,8
9,35 - 40,isro match point,20


### Venue wise stats

In [26]:
df.groupby("venue").agg(**{
    "total_games": pd.NamedAgg("date", "count"), 
    "average_ppg": pd.NamedAgg("total_points_per_game", "mean"),
    **{f"{fn}_margin": pd.NamedAgg("margin", fn) for fn in ["mean", "max", "min"]}
})

,total_games,average_ppg,mean_margin,max_margin,min_margin
venue,,,,,
infinity badminton academy,16,37.437500,4.687500,8,2
isro match point,32,38.343750,4.781250,11,2
match point - gublaala,30,37.866667,4.466667,8,2
prakash badminton academy,10,37.500000,5.500000,11,2


In [27]:
px.bar(
    df.groupby("venue").agg(**{
        "total_games": pd.NamedAgg("date", "count"),
        "average_ppg": pd.NamedAgg("total_points_per_game", "mean"),
        **{f"{fn}_margin": pd.NamedAgg("margin", fn) for fn in ["mean", "max", "min"]}
    }).reset_index(),
    y="venue",
    x="average_ppg",
    orientation='h'
)

### Datewise stats

In [28]:
date_df = df.groupby("date").agg(**{
    "total_games": pd.NamedAgg("date", "count"),
    "average_ppg": pd.NamedAgg("total_points_per_game", "mean"),
})

date_df['games_avg_5d'] = date_df['total_games'].rolling(5).mean()
date_df['ppg_avg_5d'] = date_df['average_ppg'].rolling(5).mean()
date_df

,total_games,average_ppg,games_avg_5d,ppg_avg_5d
date,,,,
1/3/2023,16,37.437500,NaN,NaN
1/4/2023,8,38.000000,NaN,NaN
1/5/2023,10,38.000000,NaN,NaN
12/24/2022,2,38.000000,NaN,NaN
12/26/2022,18,36.777778,10.8,37.643056
12/27/2022,9,37.777778,9.4,37.711111
12/28/2022,6,42.500000,9.0,38.611111
12/29/2022,9,38.444444,8.8,38.700000
12/30/2022,10,37.500000,10.4,38.600000


In [29]:
px.bar(
    date_df,
    x=date_df.index,
    y="total_games",
    template="plotly_white",
    color_discrete_sequence=px.colors.sequential.Bluyl_r,
)

In [30]:
date_df['total_games'].max(), date_df['total_games'].idxmax()

(18, '12/26/2022')

### Player Stats

In [31]:
player = "raghotham"

In [32]:
player_matches = df[
    np.where(
        np.logical_or.reduce([df[i] == player for i in ["team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2"]]),
        True,
        False
    )
].copy()

In [33]:
player_matches["belongs_to"] = np.where(
    np.logical_or(
        *[player_matches[i] == player for i in ["team_1_player_1", "team_1_player_2"]]
    ),
    'team_1',
    'team_2'
)

player_matches['player_team_points'] = np.where(
    player_matches["belongs_to"] == 'team_1',
    player_matches["points_team_1"],
    player_matches["points_team_2"]
)

#### Player Win - Loss stats

In [34]:
player_matches['result'] = np.where(player_matches.belongs_to == player_matches.winner, "win", "loss")
player_win_loss_df = player_matches.groupby("result").agg(**{
    "total_games": pd.NamedAgg("result", "count"), 
    "average_ppg": pd.NamedAgg("player_team_points", "mean"),
    **{f"{fn}_margin": pd.NamedAgg("margin", fn) for fn in ["mean", "max", "min"]}
})

In [35]:
player_win_loss_df

,total_games,average_ppg,mean_margin,max_margin,min_margin
result,,,,,
loss,36,17.444444,4.222222,11,2
win,45,21.111111,5.177778,11,2


In [36]:
px.pie(
    player_win_loss_df.reset_index(),
    values="total_games",
    names=player_win_loss_df.index,
    template="plotly_white",
    color="result",
    color_discrete_map={"win":'#b5de2b', "loss":'lightslategrey'},
    hole=0.3,
)

#### player - partner stats

In [37]:
player_matches['partner'] = np.where(
    player_matches["belongs_to"] == 'team_1',
    np.where(
        player_matches["team_1_player_1"] == player,
        player_matches["team_1_player_2"],
        player_matches["team_1_player_1"]
    ),
    np.where(
        player_matches["team_2_player_1"] == player,
        player_matches["team_2_player_2"],
        player_matches["team_2_player_1"]
    ),
)

player_matches['is_win'] = np.where(player_matches['result'] == 'win', 1, 0)

In [38]:
player_partner_stats = player_matches.groupby(["partner"]).agg(**{
    "total_games": pd.NamedAgg("result", "count"), 
    "wins": pd.NamedAgg("is_win", "sum"),
    "average_ppg": pd.NamedAgg("player_team_points", "mean"),
    **{f"{fn}_margin": pd.NamedAgg("margin", fn) for fn in ["mean", "max", "min"]}
})
player_partner_stats["win_pct"] = round(player_partner_stats['wins'] * 100 / player_partner_stats['total_games'], 2)

In [39]:
player_partner_stats

,total_games,wins,average_ppg,mean_margin,max_margin,min_margin,win_pct
partner,,,,,,,
ajay,20,16,20.250000,5.600000,11,2,80.00
nithin,24,7,18.000000,4.708333,11,2,29.17
prateek,3,2,20.666667,4.666667,9,2,66.67
sameer,3,1,19.666667,4.000000,7,2,33.33
srinidhi,12,7,20.083333,4.333333,8,2,58.33
vatsa,3,2,20.333333,3.333333,6,2,66.67
vinay,16,10,19.875000,4.500000,10,2,62.50


In [40]:
px.colors.sequential.Bluyl_r

['rgb(4, 82, 117)',
 'rgb(0, 113, 139)',
 'rgb(8, 144, 153)',
 'rgb(70, 174, 160)',
 'rgb(124, 203, 162)',
 'rgb(183, 230, 165)',
 'rgb(247, 254, 174)']

In [41]:
partner_list = player_partner_stats.index.to_list()
bar_colors = ['lightslategrey' for i in range(player_partner_stats.shape[0])]
bar_colors[partner_list.index(player_partner_stats['win_pct'].idxmax())] = '#b5de2b'

In [42]:
fig = go.Figure(
    go.Bar(
        y=player_partner_stats.index,
        x=player_partner_stats['win_pct'],
        orientation='h',
        marker_color=bar_colors,
        hovertemplate="Win Percentage: %{x} %",
#         hovertext=player_partner_stats['win_pct']
    )
)
fig.update_layout(
    plot_bgcolor="white",
    title_text="Partnerwise win percentages"
)

#### player - datewise performance

In [44]:
daily_performance = player_matches.groupby(["date", "result"]).agg(**{
    "total_games": pd.NamedAgg("result", "count"), 
    "average_ppg": pd.NamedAgg("player_team_points", "mean"),
    **{f"{fn}_margin": pd.NamedAgg("margin", fn) for fn in ["mean", "max", "min"]}
}).reset_index()

In [269]:
daily_performance

total_games  average_ppg  mean_margin  max_margin  \
date       result                                                      
12/24/2022 win               2    21.000000     4.000000           6   
12/26/2022 loss              7    17.142857     4.142857           6   
           win              11    21.000000     6.272727          11   
12/27/2022 loss              3    15.333333     5.666667           6   
           win               6    21.333333     4.166667          10   
12/28/2022 loss              4    20.500000     3.250000           7   
           win               2    21.000000     3.000000           3   
12/29/2022 loss              2    21.000000     2.000000           2   
           win               7    21.142857     5.428571          10   
12/30/2022 loss              7    16.000000     5.428571          11   
           win               3    21.666667     5.666667           8   

                   min_margin  
date       result              
12/24/2022 win              2  
12/26/2022 loss             2  
           win              3  
12/27/2022 loss             5  
           win              2  
12/28/2022 loss             2  
           win              3  
12/29/2022 loss             2  
           win              2  
12/30/2022 loss             2  
           win              2

In [58]:
daily_performance_res_ignored = player_matches.groupby(["date"]).agg(**{
    "total_games": pd.NamedAgg("result", "count"), 
    "wins": pd.NamedAgg("is_win", "sum"),
    "average_ppg": pd.NamedAgg("player_team_points", "mean")
}).reset_index()
daily_performance_res_ignored['win_pct'] = round(daily_performance_res_ignored['wins'] * 100 / daily_performance_res_ignored['total_games'], 2)
daily_performance_res_ignored["average_ppg"] = round(daily_performance_res_ignored["average_ppg"], 2)

In [86]:
daily_win_pct = go.Bar(
    x=daily_performance_res_ignored["date"],
    y=daily_performance_res_ignored["win_pct"],
    marker=dict(color="#b5de2b"),
    width=0.05
)

daily_win_pct_fig = go.Figure(daily_win_pct)
daily_win_pct_fig.update_layout(plot_bgcolor="#f9f9ff")

In [97]:
daily_win_pct = go.Bar(
    x=list(range(100)),
    y=np.random.rand(100),
    marker=dict(color="#b5de2b"),
    width=0.1
)

daily_win_pct_fig = go.Figure(daily_win_pct)
daily_win_pct_fig.update_layout(plot_bgcolor="#f9f9ff")

In [98]:
daily_win_pct = go.Scatter(
    x=list(range(100)),
    y=np.random.rand(100),
    line_color="#b5de2b",
    fill="tozeroy"
)

daily_win_pct_fig = go.Figure(daily_win_pct)
daily_win_pct_fig.update_layout(plot_bgcolor="#f9f9ff")

#### Player Venue wise stats

In [207]:
player_matches.groupby(["venue", "is_win"]).agg(**{
    "total_games": pd.NamedAgg("is_win", "count"), 
    "average_ppg": pd.NamedAgg("player_team_points", "mean"),
    **{f"{fn}_margin": pd.NamedAgg("margin", fn) for fn in ["mean", "max", "min"]}
})

total_games  average_ppg  mean_margin  \
venue                     is_win                                          
isro match point          False            11    18.454545     3.909091   
                          True             21    21.142857     5.238095   
match point - gublaala    False             5    17.400000     4.000000   
                          True              7    21.000000     5.142857   
prakash badminton academy False             7    16.000000     5.428571   
                          True              3    21.666667     5.666667   

                                  max_margin  min_margin  
venue                     is_win                          
isro match point          False            7           2  
                          True            11           2  
match point - gublaala    False            6           2  
                          True             8           2  
prakash badminton academy False           11           2  
                          True             8           2

In [77]:
# df[i] for i in ["team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2"]
# pd.unique(df[["team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2"]])
# pd.concat([df[i] for i in ["team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2"]]).unique()
list(np.unique(df[["team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2"]].values))

['aakarsh',
 'ajay',
 'nayana',
 'nithin',
 'prateek',
 'raghotham',
 'sameer',
 'shankar',
 'srinidhi',
 'swaroop',
 'vatsa',
 'vinay']

In [109]:
df2 = df.copy()
df2['point_bins'] = pd.cut(
    df2["total_points_per_game"],
    [0, 30, 33, 36, 39, 42, 45, 48, 51, float("inf")],
    labels=['< 30', '30 - 33', '33 - 36', '36 - 39', '36 - 42', '42 - 45', '45 - 48', '48 - 51', "> 51"]
)

In [101]:
df2.groupby("point_bins").agg({"point_bins": "count"})

,point_bins
point_bins,
"(0.0, 30.0]",0
"(30.0, 32.0]",4
"(32.0, 34.0]",8
"(34.0, 36.0]",28
"(36.0, 38.0]",8
"(38.0, 40.0]",26
"(40.0, 42.0]",7
"(42.0, 44.0]",4
"(44.0, 46.0]",1


In [116]:
df["total_points_per_game"].mean()

37.92045454545455

In [129]:
bin_df = df2.groupby("point_bins").agg({"point_bins": "count"})
bin_chart = go.Scatter(
    x=bin_df.index,
    y=bin_df["point_bins"],
    line_color="mediumturquoise",
    fill="tozeroy",
    line_shape="spline"
)

go.Figure(bin_chart).update_layout(plot_bgcolor="white", width=600, height=300, margin=dict(t=0, b=0))

In [146]:
player_matches

,date,team_1_player_1,team_1_player_2,team_2_player_1,team_2_player_2,points_team_1,points_team_2,winner,margin,belongs_to,is_win,partner,player_team_points
0,12/27/2022,vinay,raghotham,ajay,aakarsh,15,21,team_2,6,team_2,True,aakarsh,21
1,12/28/2022,vinay,raghotham,ajay,sameer,25,27,team_2,2,team_2,True,sameer,27
2,12/28/2022,vinay,raghotham,ajay,srinidhi,21,18,team_1,3,team_2,False,srinidhi,18
4,12/28/2022,sameer,raghotham,ajay,srinidhi,21,18,team_1,3,team_2,False,srinidhi,18
5,12/28/2022,sameer,raghotham,ajay,vinay,24,26,team_2,2,team_2,True,vinay,26
6,12/28/2022,srinidhi,raghotham,ajay,vinay,19,21,team_2,2,team_2,True,vinay,21
7,12/29/2022,srinidhi,raghotham,ajay,prateek,21,16,team_1,5,team_2,False,prateek,16
8,12/29/2022,srinidhi,raghotham,ajay,prateek,22,24,team_2,2,team_2,True,prateek,24
9,12/29/2022,srinidhi,raghotham,ajay,prateek,22,20,team_1,2,team_2,False,prateek,20
10,12/29/2022,prateek,raghotham,ajay,srinidhi,21,18,team_1,3,team_2,False,srinidhi,18
